# Difference method

In this model, we will use what we called the Difference method. We first calculate the difference between the experimental band gap and the PBE band gap, and we then evaluate the error on this difference.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.vanilla import MODNetModel
from modnet.hyper_opt.fit_genetic import FitGenetic
import numpy as np
import os
import copy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [4]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.reindex([ids])
    data.df_targets = data.df_targets.reindex([ids])
    data.df_structure = data.df_structure.reindex([ids])
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    #data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

def MD_append(md,lmd):
    md = copy.deepcopy(md)
    for m in lmd:
        md.df_structure.append(m.df_structure)
        md.df_targets.append(m.df_targets)
        md.df_featurized.append(m.df_featurized)
    return md

In [5]:
md_exp = MODData.load('exp_gap_all_mpid')
md_exp.df_targets.columns = ['exp_gap','mp_id']
md_pbe = MODData.load('pbe_gap.zip')
md_pbe.df_targets.columns = ['gap']
md_joint = MODData.load('exp_pbe_joint')


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766

2021-06-14 17:44:29,521 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f3f28068970> object, created with modnet version <=0.1.7
2021-06-14 17:44:37,814 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f3eb95a29d0> object, created with modnet version <=0.1.7
2021-06-14 17:44:38,912 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f3f280680d0> object, created with modnet version <=0.1.7


In [6]:
md_joint.df_targets

,exp_gap,pbe_gap,difference
mp-12699,0.00,0.0000,0.0000
mp-559459,3.40,2.1704,1.2296
mp-21162,0.00,0.0000,0.0000
mp-1306,0.00,0.0000,0.0000
mp-15252,0.53,0.1049,0.4251
...,...,...,...
mp-556541,1.80,1.5135,0.2865
mp-2602,1.04,0.0000,1.0400
mp-30366,0.00,0.0000,0.0000
mp-3718,1.45,0.2418,1.2082


In [7]:
md_joint.df_targets['difference'] = md_joint.df_targets['exp_gap'] - md_joint.df_targets['pbe_gap']

In [8]:
md_joint.df_targets

,exp_gap,pbe_gap,difference
mp-12699,0.00,0.0000,0.0000
mp-559459,3.40,2.1704,1.2296
mp-21162,0.00,0.0000,0.0000
mp-1306,0.00,0.0000,0.0000
mp-15252,0.53,0.1049,0.4251
...,...,...,...
mp-556541,1.80,1.5135,0.2865
mp-2602,1.04,0.0000,1.0400
mp-30366,0.00,0.0000,0.0000
mp-3718,1.45,0.2418,1.2082


In [9]:
md_joint.df_structure

,structure
id,
mp-12699,"[[ 0.13616363 -2.60631371 1.50340165] Ca, [-1..."
mp-559459,"[[2.14049282 6.43464803 1.76199846] Cs, [-0.70..."
mp-21162,"[[0. 0. 0.] Sr, [0. 2.5259875 2.5259875..."
mp-1306,"[[3.259129 1.88166243 1.15971025] Th, [ 3.25..."
mp-15252,"[[0.95531657 1.042239 0.86229681] Cu, [2.803..."
...,...
mp-556541,"[[0. 6.49363948 5.82657217] In, [ 0. ..."
mp-2602,"[[3.023318 3.023318 3.023318] Nd, [0. 0. 0.] As]"
mp-30366,"[[4.51774558 1.37529211 4.32659015] Ca, [1.109..."


In [10]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(md_joint.df_targets['exp_gap'], md_joint.df_targets['pbe_gap'])

0.429783975697596

In [11]:
md_joint.df_targets['difference'].mad()

0.5494667780829975

In [12]:
md_joint.df_targets = md_joint.df_targets.drop(columns=['exp_gap', 'pbe_gap'])

In [14]:
import uncertainty_toolbox as uct

k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
errors = np.ones((int(len(md_joint.df_targets)/k),5))
maes = np.ones(5)
uncertainties = np.ones(5)
metricss = []
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_joint_exp-pbe_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
       
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    # difference prediction
    ga = FitGenetic(train)
    model = ga.run(size_pop=20, num_generations=10, n_jobs=10)
    
    pred, std = model.predict(test, return_unc=True)
    true = test.df_targets
    metrics = uct.metrics.get_all_metrics(pred['difference'].values, std['difference'].values, true['difference'].values)
    error = pred-true
    error = error['difference'].drop(pred.index[((pred['difference']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    std = std['difference'].drop(std.index[((std['difference']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    uncertainty = np.abs(std).mean()
    print('MAE =')
    print(mae)
    print('uncertainty =')
    print(uncertainty)
    maes[i] = mae
    uncertainties[i] = uncertainty
    metricss.append(metrics)

2021-06-15 00:06:47,030 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-15 00:06:47,059 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-15 00:06:47,073 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-15 00:06:47,095 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-15 00:06:47,112 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-15 00:06:47,151 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-15 00:06:47,168 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-15 00:06:47,219 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-15 00:06:47,235 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-15 00:06:47,281 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-15 00:06:47,858 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f3ea6d8e970> object, created with modnet version 0.1.9
2021-06-15 00

100%|██████████| 100/100 [17:12<00:00, 10.32s/it]

2021-06-15 00:24:03,746 - modnet - INFO - Loss per individual: ind 0: 0.939 	ind 1: 0.931 	ind 2: 0.888 	ind 3: 0.818 	ind 4: 0.872 	ind 5: 1.012 	ind 6: 0.927 	ind 7: 0.939 	ind 8: 1.204 	ind 9: 0.957 	ind 10: 0.921 	ind 11: 1.052 	ind 12: 0.936 	ind 13: 0.957 	ind 14: 0.988 	ind 15: 1.060 	ind 16: 1.088 	ind 17: 0.894 	ind 18: 0.881 	ind 19: 0.999 	


2021-06-15 00:24:05,297 - modnet - INFO - Generation number 1
2021-06-15 00:24:08,483 - modnet - INFO - Multiprocessing on 10 cores. Total of 128 cores available.


100%|██████████| 100/100 [16:32<00:00,  9.93s/it]

...

 (3/n) Calculating adversarial group calibration metrics
  [1/2] for mean absolute calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


 20%|██        | 2/10 [00:00<00:00, 11.03it/s]

  [2/2] for root mean squared calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


100%|██████████| 10/10 [00:00<00:00, 10.53it/s]


 (4/n) Calculating sharpness metrics
 (n/n) Calculating proper scoring rule metrics
**Finished Calculating All Metrics**


===================== Accuracy Metrics =====================
  MAE           0.198
  RMSE          0.421
  MDAE          0.008
  MARPD         156.454
  R2            0.471
  Correlation   0.724
=============== Average Calibration Metrics ================
  Root-mean-squared Calibration Error   0.113
  Mean-absolute Calibration Error       0.098
  Miscalibration Area                   0.099
========== Adversarial Group Calibration Metrics ===========
  Mean-absolute Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.170
     Group Size: 0.56 -- Calibration Error: 0.116
     Group Size: 1.00 -- Calibration Error: 0.098
  Root-mean-squared Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.182
     Group Size: 0.56 -- Calibration Error: 0.134
     Group Size: 1.00 -- Calibration Error: 0.113
==============

In [15]:
maes

array([0.33880384, 0.2919107 , 0.32583968, 0.27448574, 0.19796235])

In [16]:
maes.mean()

0.2858004611596556

In [17]:
uncertainties

array([0.13591428, 0.17899893, 0.14358331, 0.13451155, 0.13358939])

In [18]:
uncertainties.mean()

0.14531949162483215

In [19]:
metricss

[{'accuracy': {'mae': 0.33880383734038305,
   'rmse': 0.6863446648592074,
   'mdae': 0.06272993981838226,
   'marpd': 148.13482658132244,
   'r2': 0.38366277611131727,
   'corr': 0.6381710485850238},
  'avg_calibration': {'rms_cal': 0.10135241924598293,
   'ma_cal': 0.08396220267188012,
   'miscal_area': 0.08480017864182088},
  'adv_group_calibration': {'ma_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
           0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
    'adv_group_cali_mean': array([0.43801515, 0.13527645, 0.11223977, 0.10264779, 0.10101123,
           0.09757431, 0.0934084 , 0.0892574 , 0.08845365, 0.0839622 ]),
    'adv_group_cali_stderr': array([0.04633965, 0.02141775, 0.01206075, 0.0045981 , 0.00586519,
           0.00445835, 0.0036416 , 0.00254372, 0.00143897, 0.        ])},
   'rms_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
           0.55555556, 0

#### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: 18%